# JPL Ephemeris calculation and orbit propagation for LSST 

In [1]:
# This is an example notebook presenting the use of the f2py wrapped ssd module kindly made available to LSST by the NASA Jet Propulsion Laboratory, California Institute of Technology 
# The ssd module is under JPL/CalTech license (distribution and use limited to LSST)
# written by S. Eggl 20190720 LSST/University of Washington

In [1]:
import numpy as np
import json
import requests
import pandas as pd

In [2]:
import ssd

In [3]:
#import ssd_unit_tests

In [4]:
!pwd

/astro/users/eggl/JPL/JPLTOOLS/jpl-libraries/test


In [5]:
obs_code='807'
ut0 = np.array([58468.0,58568.0,58668.0,58768.0])
cov_int=1
n_eph=len(ut0)
Tref=2400000.5

In [6]:
#TARGET
tname='Siegfried'

In [7]:
url="https://ssd-api.jpl.nasa.gov/sbdb.api?sstr="+tname+"&cov=src&phys-par=true&full-prec=true"

In [8]:
r=requests.request("GET",url)

In [9]:
#r.text

In [10]:
ast=json.loads(r.text)

In [11]:
#spectral slope parameter G
g_param=0.15

#absolute magnitude H
h_v=ast['phys_par'][0]['value']

#epoch of orbital elements at Tref [MJD]
epoch_mjd=float(ast['orbit']['epoch'])-Tref

#orbital elements
elem=ast['orbit']['elements']
hdr=[]
val=[]
for i in range(len(elem)):
    hdr.append(elem[i]['name'])
    val.append(elem[i]['value'])

#pic cometary orbital elements: e,q[au],tp[MJD],node[deg],peri[deg],inc[deg]
idx=[0,2,7,4,5,3]
elements=[]
for i in idx:
    if(elem[i]['name']=='tp'):
            elements.append(float(elem[i]['value'])-Tref)
    else:
            elements.append(float(elem[i]['value']))

#square root covariance matrix for cometary orbital elements
src=ast['orbit']['covariance']['data']


In [12]:
#absolute magnitude H
h_v

'12.8'

In [13]:
#epoch of osculating cometary orbital elements
epoch_mjd

58600.0

In [14]:
hdr

['e', 'a', 'q', 'i', 'om', 'w', 'ma', 'tp', 'per', 'n', 'ad']

In [15]:
elements

[0.007179709583604317,
 3.224300394965053,
 59019.99074555654,
 57.94736169556705,
 297.060245083253,
 8.94174142498718]

In [16]:
src

['-7.30758692296977E-9',
 '4.382705220637333E-8',
 '-1.50567315673694E-7',
 '-1.580393656215238E-8',
 '4.990217319865802E-8',
 '-2.144863132279895E-5',
 '-1.522924868782715E-9',
 '4.310160088087949E-9',
 '-.0001765466699452076',
 '-5.220128811753229E-7',
 '-1.227256440856608E-8',
 '3.471999146585647E-8',
 '-.001280090122935523',
 '7.088113260704621E-8',
 '-3.807627393885189E-6',
 '-2.723681462245841E-10',
 '7.394772596575109E-10',
 '1.160018565127471E-5',
 '8.240009079701517E-8',
 '-4.736490292424078E-8',
 '-8.370061313329297E-8']

In [17]:
ssd.ssd.ssd_init()

In [18]:
#call f2py wrapped SSD ephemeris function 
[ra, dec, err_ra, err_dec, smaa, smia,pa, v_minus_h, errcod]=ssd.ssd.ssd_compute_n_ephemerides(epoch_mjd,elements, src, obs_code, g_param, ut0, cov_int)

In [19]:
#output times
ut0

array([58468., 58568., 58668., 58768.])

In [20]:
#ephemerides (RA, DEC) [deg]
radec=np.rad2deg(np.array([ra,dec]))

In [21]:
#3 sigma error in RA and DEC [arcseconds?]
err=3*np.array([err_ra,err_dec])

In [22]:
#3 sigma semimajor and semiminor axis of skyplane ellipsiod
ell=3*np.array([smaa, smia])

In [23]:
results=pd.DataFrame(zip(ut0,radec[0],radec[1],err[0],err[1],ell[0],ell[1]),columns=['UT0','RA','DEC','3_sig_err_RA','3_sig_err_DEC','SMAA','SMIA'])

In [24]:
results

,UT0,RA,DEC,3_sig_err_RA,3_sig_err_DEC,SMAA,SMIA
0,58468.0,260.672628,-25.869765,0.065296,0.042854,0.066103,0.041598
1,58568.0,297.643401,-26.522115,0.079724,0.052317,0.079895,0.052056
2,58668.0,305.546897,-30.451137,0.121074,0.080357,0.122255,0.078549
3,58768.0,297.423988,-30.540080,0.087669,0.059135,0.088077,0.058526


In [25]:
# JPL/HORIZONS            15147 Siegfried (2000 EJ134)       2019-Jul-25 00:42:43
# Rec #:   15147 (+COV) Soln.date: 2019-May-30_02:18:44   # obs: 1473 (1986-2019)
 
# IAU76/J2000 helio. ecliptic osc. elements (au, days, deg., period=Julian yrs):
 
#   EPOCH=  2456624.5 ! 2013-Nov-28.00 (TDB)         Residual RMS= .31963
#    EC= .01172746704294454  QR= 3.209039705486225   TP= 2456833.5581635539
#    OM= 58.01315850610064   W=  288.8518457756886   IN= 8.940294161750789
#    A= 3.247120200623517    MA= 324.7853272565625   ADIST= 3.285200695760809
#    PER= 5.85135            N= .168444375           ANGMOM= .030995651
#    DAN= 3.23442            DDN= 3.25902            L= 347.0802137
#    B= -8.457096200000001   MOID= 2.21706009        TP= 2014-Jun-25.0581635539
 
# Asteroid physical parameters (km, seconds, rotational period in hours):
#    GM= n.a.                RAD= 9.051              ROTPER= n.a.
#    H= 12.8                 G= .150                 B-V= n.a.
#                            ALBEDO= .037            STYP= n.a.
 
# ASTEROID comments: 
# 1: soln ref.= JPL#25, OCC=0
# 2: source=ORB

# Results

# *******************************************************************************
# Ephemeris / WWW_USER Thu Jul 25 00:42:44 2019 Pasadena, USA      / Horizons    
# *******************************************************************************
# Target body name: 15147 Siegfried (2000 EJ134)    {source: JPL#25}
# Center body name: Earth (399)                     {source: DE431}
# Center-site name: Cerro Tololo Observatory, La Serena
# *******************************************************************************
# Start time      : A.D. 2018-Dec-16 00:00:00.0000 UT      
# Stop  time      : A.D. 2019-Oct-12 00:00:00.0000 UT      
# Step-size       : 144000 minutes
# *******************************************************************************
# Target pole/equ : No model available
# Target radii    : 9.1 km                                                       
# Center geodetic : 289.194100,-30.169117,2.3888790 {E-lon(deg),Lat(deg),Alt(km)}
# Center cylindric: 289.194100,5520.92438,-3187.796 {E-lon(deg),Dxy(km),Dz(km)}
# Center pole/equ : High-precision EOP model        {East-longitude positive}
# Center radii    : 6378.1 x 6378.1 x 6356.8 km     {Equator, meridian, pole}    
# Target primary  : Sun
# Vis. interferer : MOON (R_eq= 1737.400) km        {source: DE431}
# Rel. light bend : Sun, EARTH                      {source: DE431}
# Rel. lght bnd GM: 1.3271E+11, 3.9860E+05 km^3/s^2                              
# Small-body perts: Yes                             {source: SB431-N16}
# Atmos refraction: NO (AIRLESS)
# RA format       : DEG
# Time format     : JD  
# EOP file        : eop.190724.p191015                                           
# EOP coverage    : DATA-BASED 1962-JAN-20 TO 2019-JUL-24. PREDICTS-> 2019-OCT-14
# Units conversion: 1 au= 149597870.700 km, c= 299792.458 km/s, 1 day= 86400.0 s 
# Table cut-offs 1: Elevation (-90.0deg=NO ),Airmass (>38.000=NO), Daylight (NO )
# Table cut-offs 2: Solar elongation (  0.0,180.0=NO ),Local Hour Angle( 0.0=NO )
# Table cut-offs 3: RA/DEC angular rate (     0.0=NO )                           
# *******************************************************************************
# Initial IAU76/J2000 heliocentric ecliptic osculating elements (au, days, deg.):
#   EPOCH=  2456624.5 ! 2013-Nov-28.00 (TDB)         Residual RMS= .31963        
#    EC= .01172746704294454  QR= 3.209039705486225   TP= 2456833.5581635539      
#    OM= 58.01315850610064   W=  288.8518457756886   IN= 8.940294161750789       
#   Equivalent ICRF heliocentric equatorial cartesian coordinates (au, au/d):
#    X= 2.072641281593817E+00  Y=-2.023372080868738E+00  Z=-1.397785215368724E+00
#   VX= 7.216568830329465E-03 VY= 6.019215773862436E-03 VZ= 2.139021171685928E-03
# Asteroid physical parameters (km, seconds, rotational period in hours):        
#    GM= n.a.                RAD= 9.051              ROTPER= n.a.                
#    H= 12.8                 G= .150                 B-V= n.a.                   
#                            ALBEDO= .037            STYP= n.a.                  
# **************************************************************************************************************************************
# Date_________JDUT     R.A._(ICRF/J2K)_DEC dRA*cosD d(DEC)/dt  APmag  S-brt RA_3sigma DEC_3sigma SMAA_3sig SMIA_3sig    Theta Area_3sig
# **************************************************************************************************************************************
# $$SOE
# 2458468.500000000 Cm  260.67263 -25.86976 54.55297  -6.05924  18.65   7.24     0.072      0.043     0.073     0.041  -11.457      0.02
# 2458568.500000000 A   297.64340 -26.52211 39.16106  1.265288  18.93   7.95     0.094      0.052     0.095     0.052    5.383      0.03
# 2458668.500000000     305.54661 -30.45119 -22.2438  -12.3865  17.64   7.58     0.149      0.081     0.151     0.078    9.760      0.07
# 2458768.500000000 Am  297.41694 -30.54095 20.10472  9.578836  18.65   7.98     0.111      0.059     0.111     0.058    4.844      0.04
# $$EOE
# **************************************************************************************************************************************
# Column meaning:
 
# TIME

#   Times PRIOR to 1962 are UT1, a mean-solar time closely related to the
# prior but now-deprecated GMT. Times AFTER 1962 are in UTC, the current
# civil or "wall-clock" time-scale. UTC is kept within 0.9 seconds of UT1
# by introduction of integer leap-seconds for 1972 and later.

#   Conversion from the internal Barycentric Dynamical Time (TDB) to the
# non-uniform civil UT time-scale requested for output has not been determined
# for UTC times after the next July or January 1st. Therefore, the last known
# leap-second is used as a constant over future intervals.

#   Time tags refer to the UT time-scale conversion on Earth regardless of
# where the observer is located in the solar system. For example, if an
# observation from the surface of another body has an output time-tag of
# 12:31:00 UT, it refers to a time-scale conversion from TDB to UT valid
# at the center of the Earth at that instant, not the observer's location
# elsewhere in the solar system, where clock rates may differ slightly due
# to the local gravity field and there is no precisely defined or adopted
# "UT" analog.

#   Any 'b' symbol in the 1st-column denotes a B.C. date. First-column blank
# (" ") denotes an A.D. date. Calendar dates prior to 1582-Oct-15 are in the
# Julian calendar system. Later calendar dates are in the Gregorian system.

#   NOTE: "n.a." in output means quantity "not available" at the print-time.
 
# SOLAR PRESENCE (OBSERVING SITE)
#   Time tag is followed by a blank, then a solar-presence symbol:

#         '*'  Daylight (refracted solar upper-limb on or above apparent horizon)
#         'C'  Civil twilight/dawn
#         'N'  Nautical twilight/dawn
#         'A'  Astronomical twilight/dawn
#         ' '  Night OR geocentric ephemeris

# LUNAR PRESENCE (OBSERVING SITE)
#   The solar-presence symbol is immediately followed by a lunar-presence symbol:

#         'm'  Refracted upper-limb of Moon on or above apparent horizon
#         ' '  Refracted upper-limb of Moon below apparent horizon OR geocentric
#              ephemeris
 
# STATISTICAL UNCERTAINTIES

#   Output includes formal +/- 3 standard-deviation statistical orbit uncertainty
# quantities. There is a 99.7% chance the actual value is within given bounds.
# These statistical calculations assume observational data errors are random. If
# there are systematic biases (such as timing, reduction or star-catalog errors),
# results can be optimistic. Because the epoch covariance is mapped using
# linearized variational partial derivatives, results can also be optimistic for
# times far from the solution epoch, particularly for objects having close
# planetary encounters.
 
#  R.A._(ICRF/J2K)_DEC =
#    Astrometric right ascension and declination of the TARGET CENTER with
# respect to the observing site in the coordinates of the ICRF/J2000 inertial
# reference frame. Compensated for down-leg light-time.
#    Units: DEGREES and DEGREES
 
#  dRA*cosD d(DEC)/dt =
#     The rate of change of target center apparent RA and DEC (airless).
# d(RA)/dt is multiplied by the cosine of the declination.
#     Units: ARCSECONDS PER HOUR
 
#  APmag S-brt =
#    Asteroid's approximate apparent visual magnitude & surface brightness:
#    APmag = H + 5*log10(delta) + 5*log10(r) - 2.5*log10((1-G)*phi1 + G*phi2)
# For solar phase angles > 90 deg, the error could exceed 1 magnitude. For
# phase angles > 120 degrees, output values are rounded to the nearest integer
# to indicate the errors could be large and unknown.
#    Units: NONE & VISUAL MAGNITUDES PER SQUARE ARCSECOND
 
#  RA_3sigma DEC_3sigma =
#   Uncertainty in Right-Ascension and Declination. Output values are the formal
# +/- 3 standard-deviations (sigmas) around nominal position. Units: ARCSECONDS
 
#  SMAA_3sig SMIA_3sig   Theta Area_3sig =
#   Plane-of-sky (POS) error ellipse data. These quantities summarize the
# target's 3-dimensional 3-standard-deviation formal uncertainty volume projected
# into a reference plane perpendicular to the observer's line-of-sight.

#    SMAA_3sig = Angular width of the 3-sigma error ellipse semi-major
#                 axis in POS. Units: ARCSECONDS.

#    SMIA_3sig = Angular width of the 3-sigma error ellipse semi-minor
#                 axis in POS. Units: ARCSECONDS.

#    Theta     = Orientation angle of the error ellipse in POS; the
#                 clockwise angle from the direction of increasing RA to
#                 the semi-major axis of the error ellipse, in the
#                 direction of increasing DEC.  Units: DEGREES.

#    Area_3sig = Area of sky enclosed by the 3-sigma error ellipse.
#                 Units: ARCSECONDS ^ 2.


#  Computations by ...
#      Solar System Dynamics Group, Horizons On-Line Ephemeris System
#      4800 Oak Grove Drive, Jet Propulsion Laboratory
#      Pasadena, CA  91109   USA
#      Information: http://ssd.jpl.nasa.gov/
#      Connect    : telnet://ssd.jpl.nasa.gov:6775  (via browser)
#                   telnet ssd.jpl.nasa.gov 6775    (via command-line)
#      Author     : Jon.D.Giorgini@jpl.nasa.gov


In [88]:
elems_t=[np.append(t,ssd.ssd.ssd_propagate_elements(epoch_mjd,elements, t)[0]) for t in ut0]

In [90]:
pd.DataFrame(elems_t,columns=['UT0[MJD]','e','q[au]','tp[MJD]','node[deg]','peri[deg]','inc[deg]'])

,UT0[MJD],e,q[au],tp[MJD],node[deg],peri[deg],inc[deg]
0,58468.0,0.007543,3.225375,58990.314415,57.957275,292.000246,8.944021
1,58568.0,0.007255,3.224587,59013.043662,57.950033,295.873173,8.942190
2,58668.0,0.007021,3.223756,59033.621531,57.941599,299.391009,8.941021
3,58768.0,0.006828,3.223234,59048.115477,57.933771,301.872078,8.940454


In [91]:
results=pd.DataFrame(zip(ut0,[for i in range 6 elems_t[i]]),columns=['UT0']+hdr)

SyntaxError: invalid syntax (<ipython-input-91-044dcb7ebd0d>, line 1)